# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [5]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010 driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [7]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel. 
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [13]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [15]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton, driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [17]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [19]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [21]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [23]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [27]:
#version 1

print(return_OAIResponse("The product has good sound and good battery life.", context_user))

There is no sentiment provided in the question. To determine the sentiment, we would need an opinion or statement about the product's sound quality and battery life.


In [29]:
#version 2

print(return_OAIResponse("I think that this product's sound quality is good, but the battery life is far too short for my needs.", context_user))

Sentiment: Neutral


In [31]:
#version 3

print(return_OAIResponse("I believe that this product is a waist of time given that the battery live is waayyy tooo shoort, and on top of that the sound quality is not great to justify it.", context_user))

Sentiment: Negative


In [33]:
#version 4

print(return_OAIResponse("In my opinion, this product is a most have if you want to hear good music on the go, given that the battery life last long enough for the average session.", context_user))

Sentiment: Positive


In [35]:
context_user = [
    {'role':'system', 'content':
     """You are an expert DnD Classes information.

     Which classes are casters?
     Classes: Bard, Cleric, Druid, Sorcerer, and Wizard

     Which classes have access to the cantrip Guidance?
     Classes: Artificer, Cleric, Druid.

     Which classes are most known for following a God?
     Classes: Cleric
     """}
]
print(return_OAIResponse("Which classes are considered half-casters?", context_user))

The classes considered half-casters are the Paladin and the Ranger in Dungeons and Dragons.


In [37]:
print(return_OAIResponse("Which classes are considered non-casters?", context_user))

Non-caster classes in DnD include Barbarian, Fighter, Monk, Paladin, Ranger, and Rogue.


In [39]:
print(return_OAIResponse(" Which classes have access to the cantrip Mage Hand?", context_user))

Classes: Bard, Sorcerer, Warlock, and Wizard


In [41]:
print(return_OAIResponse("Which classes are most known for their ability to stay hidden?", context_user))

Classes: Rogue, Ranger, Monk


In [ ]:
Few Shots Report

During the testing of the Few Shots system, varios versions were evaluated for accuracy and responsiveness.

For the versions that used the same role of the product review evaluator, the responces were accurate. Even when I tried to trick it with a statement of the product instead of an oppinion it reccognized it.

Now for the version that I made creative liberties, I also evaluated the DnD version that I made.

These versions get correct answers to the questions I gave them, however some of the answers where not what I would expect, but are technically correct, like the non-caster question where it included Paladin and Ranger even due they are half-casters, but it technically correct cause they have attribute of both caster and non-casters.

In [ ]:
I learned how to use Zero-Shot, One-Shot, Few-Shots.

I even made a new context_user for a topic that I am confident in